In [1]:
import pandas as pd
import xml.etree.ElementTree as ET

In [2]:
net_xml = "seattle.net.xml"
add_xml = "seattle.add.xml"

pc = pd.read_csv('parking_space_count.csv')

In [3]:
with open(add_xml, "w") as add:
    print("""<?xml version="1.0" encoding="UTF-8"?>

<additional xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/additional_file.xsd">

    <location netOffset="-362563.54,-4832311.19" convBoundary="0.00,0.00,2064.54,1284.45" origBoundary="13.296064,43.631354,13.321826,43.642858" projParameter="+proj=utm +zone=33 +ellps=WGS84 +datum=WGS84 +units=m +no_defs"/>
    """, file=add)

In [4]:
def side_2_roadway(block_id, side, edge_oneway_dict):
    if not edge_oneway_dict[block_id]:
        if side == 'SE':
            return block_id + '-' + 'NE'
        elif side == 'NE':
            return block_id + '-' + 'NW'
        elif side == 'NW':
            return block_id + '-' + 'SW'
        elif side == 'SW':
            return block_id + '-' + 'SE'
    else: 
        return edge_oneway_dict[block_id]

In [5]:
edges = []

root = ET.parse(net_xml).getroot()
for child in root.iter('edge'):
    if child.get('function') != 'internal':
        edges.append(child.get('id'))
        
edge_oneway_dict = {}
for edge_id in edges:
    if edge_id not in edge_oneway_dict:
        # if not checked, go ahead to check oneway
        if sum(word.startswith(edge_id[:5]) for word in edges) == 2:
            edge_oneway_dict[edge_id[:5]] = 0
        else:
            assert sum(word.startswith(edge_id[:5]) for word in edges) == 1
            edge_oneway_dict[edge_id[:5]] = edge_id

In [6]:
oneway_list = ['24-01', '24-02', '24-03', '25-02', '25-03', '26-01', '26-02', '26-03', '02-24', '02-25', '02-26', '04-24', '04-25', '04-26']
oneway_side_list = ['SE', 'SE', 'SE', 'NW', 'NW', 'SE', 'SE', 'SE', 'SW', 'SW', 'SW', 'NE', 'NE', 'NE']

In [7]:
oneway_side_dict = dict(zip(oneway_list, oneway_side_list))

In [8]:
pc_oneway = pc[pc['block_id'].isin(oneway_list)].groupby(['block_id','type']).sum().reset_index()
pc_oneway.drop(columns = ['Unnamed: 0'], inplace = True)
pc_oneway['side'] =  pc_oneway['block_id'].apply(lambda x: oneway_side_dict[x])

In [9]:
pc_twoway = pc[~pc['block_id'].isin(oneway_list)]
pc_twoway.drop(columns = ['Unnamed: 0'], inplace = True)

/Users/shuyiyin/anaconda3/envs/syin/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


### merge curbsides of oneway roads

In [10]:
pc_new = pd.concat([pc_oneway, pc_twoway])

### add one curb to 03-26-SW

In [11]:
pc_new['cnt'][(pc_new['block_id'] == '03-26') & (pc_new['side'] == 'SW') & (pc_new['type']=='PS')] = 2

<ipython-input-11-0517feea2c78>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pc_new['cnt'][(pc_new['block_id'] == '03-26') & (pc_new['side'] == 'SW') & (pc_new['type']=='PS')] = 2


In [12]:
for index, row in pc_new.iterrows():
    block_id = row['block_id']
    side = row['side']
    space_type = row['type']
    length = row['length']
    count = row['cnt']
    lane = side_2_roadway(block_id, side, edge_oneway_dict) + '_0'
    parking_area_id = str(block_id) + '-' + str(side)
    with open(add_xml, "a") as add:
        if space_type == 'PS':
            print('    <parkingArea id="{}" lane="{}" startPos="10" endPos="50" roadsideCapacity="{}" angle="0" />'.\
#                   format(parking_area_id, lane, int(length)//16), # unit of curb length?
                  format(parking_area_id, lane, count), # unit of curb length?
                  file=add)

In [13]:
with open(add_xml, "a") as add:
    print('\n</additional>', file = add)